In [5]:
from importlib import reload
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import cv2

%matplotlib inline

In [6]:
import cedar_videos
reload(cedar_videos)
from cedar_videos import *


In [7]:
measurement = "A2"